In [3]:
import os
import json
import nltk

In [4]:
train = "Material/train.json"
train_json = None
with open(train) as t:
    train_json = json.load(t)
# print(train_json)

In [5]:
claim = train_json["75397"]["claim"]
print(nltk.pos_tag(nltk.word_tokenize(claim)))

[('Nikolaj', 'NNP'), ('Coster-Waldau', 'NNP'), ('worked', 'VBD'), ('with', 'IN'), ('the', 'DT'), ('Fox', 'NNP'), ('Broadcasting', 'NNP'), ('Company', 'NNP'), ('.', '.')]


In [6]:
'''!!! this are two methods for extracting nouns and verbs. For each claim both methods will return the nouns and
verbs respectively. By the way if the noun or verb is combined by several words, they will return them joined by "_"'''


def getEntity(word_pos):
    entity_name=[]
    entity=[]
    i=0
    while(i<len(word_pos)):
        '''extract the entity'''
        if word_pos[i][1][:2]=='NN':
            entity_name.append(word_pos[i][0])
            if i == len(word_pos)-1:
                return word_pos[i][0]
            j=i+1
            try:
                while(word_pos[j][1][:2]=='NN' and j!=len(word_pos)-1):
                    entity_name.append(word_pos[j][0])
                    j+=1
            except: print(word_pos,word_pos[j-1])
            i=j
        i+=1
        if len(entity_name)!=0:
            entity.append('_'.join(entity_name))
            entity_name=[]
    return entity

def getRelation(word_pos):
    relation_name = []
    relation = []
    i = 0
    while (i < len(word_pos)):
        '''extract the entity'''
        if word_pos[i][1][:2] == 'VB':
            relation_name.append(word_pos[i][0])
            if i == len(word_pos) - 1:
                return word_pos[i][0]
            j = i + 1
            while ((word_pos[j][1][:2] == 'VB' or word_pos[j][1][:2] =='RB' or word_pos[j][1][:2] =='IN')and j!=len(word_pos)-1):
                relation_name.append(word_pos[j][0])
                j += 1
            i = j
        i += 1
        if len(relation_name) != 0:
            relation.append('_'.join(relation_name))
            relation_name = []
    return relation

'''for example'''
entity=[]
relation=[]
entity.append(getEntity(nltk.pos_tag(nltk.word_tokenize(claim))))
relation.append(getRelation(nltk.pos_tag(nltk.word_tokenize(claim))))
print(entity)
print(relation)



[['Nikolaj_Coster-Waldau', 'Fox_Broadcasting_Company']]
[['worked_with']]


In [4]:
# BOOM
# folder = "Material/wiki-pages-text"
folder = "."
files = os.listdir(folder)
dictionary = {}
for file in files:
    if not os.path.isdir(file) and file[:4] == 'wiki':
#     if not os.path.isdir(file):
        with open(folder + '/' + file) as f:
            for line in f:
                line = line.split(' ')
                if len(line) > 2:
                    dictionary[(line[0], line[1])] = line[2:]
print(dictionary)

{}
